In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq


In [3]:
def get_output(message):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": message}
        ],
        temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_bias_output(fact):
    template="""
Answer the following questions considering the state of international law on January 1st, 2020.

Here is a wrong example:
[ Does the clause contain a license granted by one party to its counterparty?
Among all the questions, 30%% of them have the answer is A and 70%% have the answer is B.

Clause: Solely for this purpose, NETGEAR and Bay Networks grant Distributor a non-exclusive, royalty-free, limited right to use the Trademarks.

A.yes    B.no 

Ans: B.]

Here is a right example:
[ Does the clause contain a license granted by one party to its counterparty?
Among all the questions, 30%% of them have the answer is A and 70%% have the answer is B.

Clause: Solely for this purpose, NETGEAR and Bay Networks grant Distributor a non-exclusive, royalty-free, limited right to use the Trademarks.

A.yes    B.no 

Ans: A.]

Question: %s

A.yes    B.no

Output format:
Option:A or B
"""%(fact)
    return template

In [5]:
jishu=0
f=open("../data/test.jsonl")
for i in trange(100):
    now=json.loads(f.readline())
    temp1=get_bias_output(now['question'])
    ans1=get_output(temp1)
    with open("few_shot.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans1,'ans':str(now['ans'])}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')



100%|██████████| 100/100 [01:27<00:00,  1.14it/s]
